# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

vacation_df=pd.read_csv('../output_data/cities.csv')
vacation_df.count()

City ID       0
City          0
Cloudiness    0
Country       0
Date          0
Humidity      0
Lat           0
Lng           0
Max Temp      0
Wind Speed    0
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"].astype(float)


# Plot Heatmap
heatmap = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
heatmap.add_layer(heat_layer)
# Show Heatmap below
heatmap

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

temperature = vacation_df.loc[(vacation_df['Max Temp']>=70)&(vacation_df['Max Temp']<=80)]
windspeed = temperature.loc[(temperature['Wind Speed']<=10)]
ideal_weather = windspeed.loc[(windspeed['Cloudiness']==0)].dropna()
ideal_weather.head()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.


hotel_df = ideal_weather
hotel_df["Hotel Name"]= ""
hotel_df

In [ ]:
import pprint

# Update parameters dictionary

params = {
    "radius": 5000,
    "types": "lodging",
#     "keyword": "hotel",
    "key": g_key}

# Use latitude and longitude to identify airports
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
# get lat, lng from hotel_df
        lat = row["Lat"]
        lng = row["Lng"]

# Change location of each iteration and leave parameters in place
        params["location"] = f"{lat},{lng}"

        
# Try:
    # Make API request to print URL
        response = requests.get(base_url, params=params).json()

# Try and except are used to skip those missing a data point. 
#     try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:

# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

heatmap.add_layer(markers)

# Display figure
heatmap